In [1]:
import pandas as pd
import os
import sklearn.datasets
import swat

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sasctl import Session
from sasctl.tasks import register_model, publish_model
from sasctl.services import model_repository as mr
from sasctl.services import model_management as mm

## Establish a session with SAS Viya

In [2]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem'
s = swat.CAS('frasepviya35smp', 8777, username='viyademo01', password='demopw', protocol='https')
Session(s)

In [3]:
project = 'Boston Housing'
model_name = 'Boston Regression'

# Fit a linear regression model using sci-kit learn

In [4]:
data = sklearn.datasets.load_boston()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.DataFrame(data.target, columns=['Price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Push python dataframe training table to cas for future performance testing in CAS

In [5]:
data_dataframe = pd.DataFrame(data.data)

In [6]:
s.upload(data_dataframe,casout="boston")

NOTE: Cloud Analytic Services made the uploaded file available as table BOSTON in caslib CASUSER(viyademo01).
NOTE: The table BOSTON has been created in caslib CASUSER(viyademo01) from binary data uploaded to Cloud Analytic Services.


[caslib]

 'CASUSER(viyademo01)'

[tableName]

 'BOSTON'

[casTable]

 CASTable('BOSTON', caslib='CASUSER(viyademo01)')

+ Elapsed: 0.00905s, user: 0.0129s, sys: 0.0136s, mem: 33.7mb

In [7]:
s.promote(name="boston", targetCaslib="PUBLIC")

NOTE: Cloud Analytic Services promoted table BOSTON in caslib CASUSER(viyademo01) to table boston in caslib PUBLIC.


+ Elapsed: 0.00138s, sys: 0.00108s, mem: 0.331mb

In [8]:
s.table.save(table=dict(name='boston', caslib='PUBLIC'), caslib='PUBLIC', name='boston.sashdat', replace=True)

NOTE: Cloud Analytic Services saved the file boston.sashdat in caslib Public.


[caslib]

 'Public'

[name]

 'boston.sashdat'

+ Elapsed: 0.00532s, user: 0.00131s, sys: 0.00125s, mem: 16.9mb

# Register the model in SAS Model Manager

In [9]:
register_model(lm,
               model_name,
               input=X_train,       # Use X to determine model inputs
               project=project,     # Register in "Iris" project
               force=True)          # Create project if it doesn't exist

<class 'sasctl.core.RestObj'>(headers={'Date': 'Tue, 21 Jul 2020 09:08:38 GMT', 'Server': 'Apache/2.4', 'Content-Security-Policy': "default-src 'self'; script-src 'self' 'unsafe-inline' 'unsafe-eval'; img-src 'self' *.sas.com blob: data:; style-src 'self' 'unsafe-inline'; child-src 'self' blob: data: mailto:;", 'Last-Modified': 'Tue, 21 Jul 2020 09:08:39 GMT', 'ETag': 'W/"1735decd899"', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'Strict-Transport-Security': 'max-age=31536000 ; includeSubDomains', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Type': 'application/vnd.sas.models.model+json;charset=UTF-8', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Vary': 'User-Agent', 'Keep-Alive': 'timeout=5, max=94', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}, data={'creationTimeStamp': '2020-07-21T09:08:38.343Z', 'modifiedTimeStamp': '2020-07-21T09:08:38.343Z', 'createdBy': 'viyademo01', 'modified

## Update SAS Model Manager project properties

In [10]:
project = mr.get_project(project)

project['function'] = 'prediction'
project['targetLevel'] = 'interval'
project['targetVariable'] = 'Price'
project['predictionVariable'] = 'var1'

project = mr.update_project(project)

# Instruct the project to look for tables in the "Public" CAS library with names starting with "boston_" and use these tables to track model performance over time.

In [11]:
mm.create_performance_definition(model_name, 'Public', 'boston')

<class 'sasctl.core.RestObj'>(headers={'Date': 'Tue, 21 Jul 2020 09:08:57 GMT', 'Server': 'Apache/2.4', 'Content-Security-Policy': "default-src 'self'; script-src 'self' 'unsafe-inline' 'unsafe-eval'; img-src 'self' *.sas.com blob: data:; style-src 'self' 'unsafe-inline'; child-src 'self' blob: data: mailto:;", 'Location': '/modelManagement/performanceTasks/1f8ed609-019a-4ff0-ad47-197193da44c5', 'ETag': '"kcvpuioj"', 'Last-Modified': 'Tue, 21 Jul 2020 09:08:55 GMT', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'Strict-Transport-Security': 'max-age=31536000 ; includeSubDomains', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Type': 'application/vnd.sas.models.performance.task+json;charset=UTF-8', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Vary': 'User-Agent', 'Keep-Alive': 'timeout=5, max=97', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}, data={'creationTimeStamp': '2020-07-21T09:08:55.

## Publish the model to the real-time scoring engine

In [12]:
module_lm = publish_model(model_name, 'maslocal')

## Select the first row of training data

In [18]:
x = X.iloc[0, :]
print(x)

CRIM         0.00632
ZN          18.00000
INDUS        2.31000
CHAS         0.00000
NOX          0.53800
RM           6.57500
AGE         65.20000
DIS          4.09000
RAD          1.00000
TAX        296.00000
PTRATIO     15.30000
B          396.90000
LSTAT        4.98000
Name: 0, dtype: float64


## Call the published module and score the record

In [21]:
result = module_lm.score(**x)
print(result)

AttributeError: 'RestObj' object has no attribute 'score'

## Build a second model

In [22]:
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

# Register the second model in Model Manager

In [23]:
model_dt = register_model(dt, 'Decision Tree', project, input=X)

# Publish from Model Manager -> MAS

In [24]:
module_dt = publish_model(model_dt, 'maslocal')

# Use MAS to score some new data

In [25]:
result = module_dt.score(**x)
print(result)

AttributeError: 'RestObj' object has no attribute 'score'

# Train a gradient boosting with SAS

In [26]:
traindata=data;
s.upload_frame(traindata,casout='boston_train')

AttributeError: startswith

In [58]:
tbl = s.CASTable('boston_train')

In [66]:
list(tbl.columns)

In [72]:
y.columns

Index(['Price'], dtype='object')

In [73]:
s.loadactionset('decisionTree')
tbl.decisiontree.gbtreetrain(target='Price'list(y.columns)'',
                             inputs=list(tbl.columns),
                             savestate='gradboost_astore')

NOTE: Added action set 'decisionTree'.
ERROR: Cannot convert parameter 'target' from value_list to string.
ERROR: The action stopped due to errors.


+ Elapsed: 9.7e-05s, user: 0.415s, sys: 0.508s

In [14]:
s.close()